### Spark Initialization

In [1]:
import findspark
findspark.init()

In [2]:
import sys
import copy
import csv
import numpy as np

from string import atoi
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
conf = SparkConf().setAppName("ContentBased")
conf = conf.setMaster("local[*]")

In [4]:
sc  = SparkContext(conf=conf)

### Load Train and Test Data

In [5]:
trainData = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_train.txt",use_unicode=False)
testData = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_test.txt",use_unicode=False)

In [6]:
train_rdd = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))
test_rdd = testData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))

Calculate average rating of users and products

In [7]:
avg_rating = train_rdd.map(lambda x: (x[0][0], x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sum(x[1])/len(x[1])))
prod_rating = train_rdd.map(lambda x: (x[0][1], x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sum(x[1])/len(x[1])))

In [8]:
train_temp = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), x[2]))

### Load product category data

In [9]:
data = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pitts_bus_20_attributes_categories.txt",use_unicode=False)

In [10]:
data.take(6)

['"f2FfutZhb4F-m1Ob0EdYaw","asian fusion caterers fast food chinese restaurants vegetarian food delivery services event planning  services food  alcohol_full_bar ambience_casual bikeparking businessacceptscreditcards businessparking_street caters goodforkids goodformeal_dinner hastv noiselevel_average restaurantsattire_casual restaurantsdelivery restaurantsgoodforgroups restaurantspricerange2_2 restaurantstableservice restaurantstakeout wheelchairaccessible "',
 '"qfcdMhm1Ff28JHVpHca20g","pizza restaurants  businessacceptscreditcards restaurantsattire_casual restaurantspricerange2_2 restaurantstakeout "',
 '"wHUru-79ExNRanJtwZEYeA",""',
 '"rreIYrzI9U052p7nN_qogA","sushi bars food thai poke japanese restaurants   ambience_casual bikeparking businessacceptscreditcards caters goodforkids goodformeal_lunch goodformeal_dinner noiselevel_average restaurantsattire_casual restaurantspricerange2_2 restaurantstakeout "',
 '"eva56motCJcevOwKzyQO1g","cafes hotels  travel swimming pools travel serv

In [11]:
train_data = data.mapPartitions(lambda x: csv.reader(x)).map(lambda x: (x[0], x[1]))

In [12]:
train_data.take(3)

[('f2FfutZhb4F-m1Ob0EdYaw',
  'asian fusion caterers fast food chinese restaurants vegetarian food delivery services event planning  services food  alcohol_full_bar ambience_casual bikeparking businessacceptscreditcards businessparking_street caters goodforkids goodformeal_dinner hastv noiselevel_average restaurantsattire_casual restaurantsdelivery restaurantsgoodforgroups restaurantspricerange2_2 restaurantstableservice restaurantstakeout wheelchairaccessible '),
 ('qfcdMhm1Ff28JHVpHca20g',
  'pizza restaurants  businessacceptscreditcards restaurantsattire_casual restaurantspricerange2_2 restaurantstakeout '),
 ('wHUru-79ExNRanJtwZEYeA', '')]

### Collect user data from train data (User, Product, Rating)

In [13]:
userReview = train_data.map(lambda x: (x[0][0], x[0][1], x[1][0][0]))

In [14]:
userReviewCollected = train_temp.map(lambda x: (x[0][0], x[0][1], x[1]))

In [15]:
userReviewCollected.take(5)

[('1VVHf1BvtGC0aSCCIjQyiA', 'K5jY2W5Q3eNnwssV5UZtow', '4'),
 ('QYKexxaOJQlseGWmc6soRg', 'rzByiKaj-bLeLz-zKNBQdg', '2'),
 ('-ARdx8hOcEWlMDjzwLYZ_g', '3cbsPfoUUrysf-M8FI_0IA', '4'),
 ('2oTG2GrBpikkonRxQywI7Q', 'CFtZH4Skp9z3o4ToSywI4w', '4'),
 ('B7aR8VlLMGzaJ7NHC-jeMQ', 'RC8NJlX3X2Eo240PU5zJWA', '4')]

### Collect product data (Product, Category Text)

In [16]:
prodReview = train_data.map(lambda x: (x[0], x[1])).groupByKey().mapValues(list)

In [17]:
prodReviewCollected = prodReview.map(lambda x: (x[0], " ".join(x[1])))

In [18]:
prodReviewCollected.take(5)

[('MvlQo4bev1eqp1q0HYOLHg',
  'professional services performing arts arts  entertainment fitness  instruction dance studios active life education  bikeparking businessparking_street byappointmentonly goodforkids wheelchairaccessible '),
 ('jemRU9DvmS0WGZuZ-k3jCQ',
  'desserts food shaved ice ice cream  frozen yogurt  bikeparking businessacceptscreditcards businessparking_lot caters restaurantspricerange2_1 restaurantstakeout '),
 ('PuMpFTKS6gY_e31UP5YVnw',
  'restaurants american_new  bikeparking businessacceptscreditcards goodforkids hastv restaurantsattire_casual restaurantsgoodforgroups restaurantspricerange2_1 restaurantstakeout '),
 ('5S8KaaAIjqo2bJcVlMNW5w',
  'chinese restaurants  businessacceptscreditcards restaurantsattire_casual restaurantsdelivery restaurantsgoodforgroups restaurantspricerange2_2 restaurantstableservice restaurantstakeout '),
 ('xZfPdAYeimiruXuGR4nSUA',
  'specialty food health markets vegetarian food restaurants   ambience_casual bikeparking businessaccepts

### TF-IDF vector creation

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

/Users/lakshya/anaconda/lib/python2.7/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [20]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,1),min_df=1, norm='l2')

Convert user and product rdd to pandas dataframe

In [21]:
spark = SparkSession(sc)

userPandas = userReviewCollected.toDF().toPandas()
prodPandas = prodReviewCollected.toDF().toPandas()

Create TF-IDF vectors on product category text

In [22]:
tfidf_prod = tf.fit_transform(prodPandas['_2'])

In [23]:
tfidf_prod.shape

(3192, 549)

Add TF-IDF vectors to product dataframe

In [24]:
prodPandas['Vector'] = tfidf_prod.toarray().tolist()

In [25]:
prodPandas

,_1,_2,Vector
0,MvlQo4bev1eqp1q0HYOLHg,professional services performing arts arts en...,"[0.0, 0.0, 0.241807631699, 0.0, 0.0, 0.0, 0.0,..."
1,jemRU9DvmS0WGZuZ-k3jCQ,desserts food shaved ice ice cream frozen yog...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,PuMpFTKS6gY_e31UP5YVnw,restaurants american_new bikeparking business...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,5S8KaaAIjqo2bJcVlMNW5w,chinese restaurants businessacceptscreditcard...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,xZfPdAYeimiruXuGR4nSUA,specialty food health markets vegetarian food ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,8A4ck_lvMLYZDrDt7w8GVA,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,P2uPCEBP2VZsApnvEHCgag,indian restaurants ambience_casual bikeparki...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,DwmAsemahJywywhuqv5J1Q,szechuan restaurants chinese ambience_casual...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,71ONxJtmDNDiJAtYHK-0RA,american_traditional restaurants diners breakf...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,_7uEyBADd8S14rp2Kzez5g,bars nightlife dive bars beer bar alcohol_ful...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Product category text not needed

In [26]:
del prodPandas['_2']

### Weighted Linear Combination of product vectors for users

In [27]:
userPandas['Vector'] = [[] for _ in range(len(userPandas))]

In [28]:
userPandas['_3'] = userPandas['_3'].astype(float)

In [29]:
for index, row in userPandas.iterrows():
    vector = np.array(prodPandas.loc[prodPandas['_1'] == row['_2'], 'Vector'].values[0])
    rating = row['_3']
    userPandas.at[index,'Vector'] = rating*vector

In [30]:
userPandas

,_1,_2,_3,Vector
0,1VVHf1BvtGC0aSCCIjQyiA,K5jY2W5Q3eNnwssV5UZtow,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,QYKexxaOJQlseGWmc6soRg,rzByiKaj-bLeLz-zKNBQdg,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,-ARdx8hOcEWlMDjzwLYZ_g,3cbsPfoUUrysf-M8FI_0IA,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,2oTG2GrBpikkonRxQywI7Q,CFtZH4Skp9z3o4ToSywI4w,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,B7aR8VlLMGzaJ7NHC-jeMQ,RC8NJlX3X2Eo240PU5zJWA,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,UsmTxWbobLsI6WR1Db0W7A,hsDkxhc-_YYsVqAeQBJJjg,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,2jKzO_01d12oiu-2bOYcYg,5REYrZfsX3m4E3FTwovp5Q,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,57tf_RmT5o3BiFj2gpV-Jg,mc5Ha4QyQZvU7fzr-ubhhw,5.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,J9OSJ7Xl9wnS0tL709AIqA,dLc1d1zwd1Teu2QED5TmlA,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,BTg_ajmMJKKpvH0P1mAUqA,oxig-TT9V2QttAvRqa5WNA,3.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Product and rating column not needed

In [31]:
del userPandas['_2']
del userPandas['_3']

Linear combination of feature vectors

In [32]:
userPandas = userPandas.groupby(['_1']).sum()

Normalize the user feature vectors

In [33]:
from sklearn.preprocessing import Normalizer

In [34]:
for index, row in userPandas.iterrows():
    vector = np.array(row['Vector']).reshape(1, -1)
    transformer = Normalizer().fit(vector)
    userPandas.at[index,'Vector'] = transformer.transform(vector)

In [35]:
userPandas

,Vector
_1,
-0-hVEpwWEcJLJoGq3rE3g,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
-2OB54nQ6FsGLUM-R1KXnA,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
-ARdx8hOcEWlMDjzwLYZ_g,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0028831..."
-Pk25bOBsvemFaWKDBVBzA,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
-Q2wBtscwW6JOqlBndji4A,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
-Q4bjWlbxmb1yKP4U7OODg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
-SDx-d5jppC4OBBosLVpYw,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
-XgVXGJnOnW0kQEol6O3Pg,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
-Y6tXYPYqeVy37-L5p0rMw,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


### Create user numpy matrix from feature vectors

In [36]:
user_matrix = np.zeros((len(userPandas), tfidf_prod.shape[1]))
idx = 0
for index, row in userPandas.iterrows():
    vector = np.array(row['Vector'])[0]
    user_matrix[idx] = vector
    idx += 1
    

In [37]:
user_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.02265067,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.00660732,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.01025369, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.0176749 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.04647705,  0.        ]])

### Create product numpy matrix from feature vectors

In [38]:
prod_matrix = np.zeros((len(prodPandas), tfidf_prod.shape[1]))
idx = 0
for index, row in prodPandas.iterrows():
    vector = np.array(row['Vector'])
    prod_matrix[idx] = vector
    idx += 1

In [39]:
prod_matrix

array([[ 0.        ,  0.        ,  0.24180763, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

### Compute cosine similarity by taking dot product

In [40]:
similarity_matrix = np.dot(user_matrix, prod_matrix.T)

In [41]:
similarity_matrix.shape

(987, 3192)

### Flatten similarity matrix to related with user and products

In [42]:
prod = prodPandas['_1'].values
user = userPandas.index.values

In [43]:
zf = similarity_matrix.flatten()
xr = np.repeat(user, prod.size)
yt = np.tile(prod, user.size)
d = np.stack((xr, yt, zf), axis=-1)

In [44]:
d.shape

(3150504, 3)

### Convert similarity matrix to RDD

In [45]:
similarity_rdd = sc.parallelize(d, 10000)

In [46]:
similarity_rdd.take(5)

[array([u'-0-hVEpwWEcJLJoGq3rE3g', u'MvlQo4bev1eqp1q0HYOLHg',
        0.08688367393230353], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'jemRU9DvmS0WGZuZ-k3jCQ',
        0.19204449974292587], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'PuMpFTKS6gY_e31UP5YVnw',
        0.4513868048288503], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'5S8KaaAIjqo2bJcVlMNW5w',
        0.3120304916702723], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'xZfPdAYeimiruXuGR4nSUA',
        0.4458662409733761], dtype=object)]

### Use Train and Test Data for predictions

In [47]:
trainData.take(5)

['1VVHf1BvtGC0aSCCIjQyiA,K5jY2W5Q3eNnwssV5UZtow,4,2016-11-16,2,2,2,past sunday one several time ive spirit its always eccentric fun time first music performance second hang recent visit sundays bingo bango spirit know pizza good drink also awesome last time get chard margarita time get tomatillo margarita hot ciders its always pleasant surprise see whats menu food drink drink little pricey drawback one coolest things spirit atmosphere its always super strange positive way really never know expect bingo bango definitely family appropriate its fun activity friends maybe even date doesnt mind something ordinary',
 'QYKexxaOJQlseGWmc6soRg,rzByiKaj-bLeLz-zKNBQdg,2,2015-04-13,0,0,0,old cramp build lot enough employees staff keep demand cause long wait time',
 '-ARdx8hOcEWlMDjzwLYZ_g,3cbsPfoUUrysf-M8FI_0IA,4,2014-03-24,6,4,3,live long world without donut menu dont know group nine din three varieties donuts include lemon lavender chocolate espresso zeppolli amaze pepper donut concoction ever m

Create key on (user, product)

In [48]:
train_rdd = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))
test_rdd = testData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))

In [49]:
similarity_rdd = similarity_rdd.map(lambda x: ((x[0], x[1]), float(x[2])))

In [50]:
test_rdd.take(5)

[(('JiPMk9WmbJu-VfTRAKpZpw', 'PdDpIGwBZoTYzOVasT-WuA'), 4.0),
 (('2wKnvn68eWybc7ID-7UQmQ', 'khRo2a5OaIjumox-tkg3GA'), 4.0),
 (('LsWpfxWjLQcazDqnZ_A62g', 'D_pwairtGGR0V_w2xx5XeA'), 2.0),
 (('0N9bSCmoJMoGmR0EldzjQg', '3iaOYhNoc6XL935MqnxJSQ'), 5.0),
 (('-hietrA8M58asfpyJkCyiA', 'O1ird5yRyuDFnOmYu90OoA'), 4.0)]

Join similarity matrix with train and test RDD to take only similarity values for training and testing

In [51]:
train = similarity_rdd.join(train_rdd)
test = similarity_rdd.join(test_rdd)

In [52]:
train.take(5)

[((u'cCke3VtFLtqgzkgYSrSE2g', u'BMR_AsSBzTQHqW-SQabI4w'),
  (0.5438565380250946, 4.0)),
 ((u'YE54kKTuqJJPNYWIKIpOEQ', u'RvwZqjdkZ_pER0moPXLZAQ'),
  (0.5721197571382111, 3.0)),
 ((u'4wp4XI9AxKNqJima-xahlg', u'ziJsGjXvidzZWC1I0-SOSg'),
  (0.32890632235941675, 3.0)),
 ((u'U92V-gp13uMZL-sl_naTHA', u'zi6cB_bkswWPLD2k3IVtyg'),
  (0.4473545937249476, 4.0)),
 ((u'rZQCd47n7OwPd71igVX6Og', u'IfHkboQZGPZkKpqHZCW4ag'),
  (0.4895856138535405, 1.0))]

Convert RDD to List

In [53]:
train_ratings = train.collect()
test_ratings = test.collect()

In [54]:
train_ratings

[((u'cCke3VtFLtqgzkgYSrSE2g', u'BMR_AsSBzTQHqW-SQabI4w'),
  (0.5438565380250946, 4.0)),
 ((u'YE54kKTuqJJPNYWIKIpOEQ', u'RvwZqjdkZ_pER0moPXLZAQ'),
  (0.5721197571382111, 3.0)),
 ((u'4wp4XI9AxKNqJima-xahlg', u'ziJsGjXvidzZWC1I0-SOSg'),
  (0.32890632235941675, 3.0)),
 ((u'U92V-gp13uMZL-sl_naTHA', u'zi6cB_bkswWPLD2k3IVtyg'),
  (0.4473545937249476, 4.0)),
 ((u'rZQCd47n7OwPd71igVX6Og', u'IfHkboQZGPZkKpqHZCW4ag'),
  (0.4895856138535405, 1.0)),
 ((u'3KkT6SmPFLGvBS1pnDBr8g', u'9DxJTxiTJQdWI85MahuWPg'),
  (0.3884902373205543, 4.0)),
 ((u'ciXjBfJrAEteIKpzZg4I9g', u'P4lfCI0WSaVoA2VFdsZRiQ'),
  (0.4719208040648843, 2.0)),
 ((u'UtkjXTiMfGRz4R1zDF3Kxg', u'kKFUNcLklJ54zZNUk-7yjQ'),
  (0.5582198365713799, 4.0)),
 ((u'DeXKbQYNx52OlOizobOLJw', u'i-2OzvZUDtvKCMq1vcRSZg'),
  (0.44314140778246924, 5.0)),
 ((u'EzVLcZ6QwLLh2oD2e_dnLg', u'hNCKIrMJSacjMiWVwRb_YQ'),
  (0.3665884384669946, 4.0)),
 ((u'MDeA-Gx8NScCyAdziq7IeA', u'Z6_gRH_m8FQlKgsNkZKmTg'),
  (0.5640442757226471, 4.0)),
 ((u'SKre6GKJAA3FLgb2e875Rg', 

### Training Regressor on similarity values

In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

Convert data to numpy array for regressor

In [56]:
X_train = []
y_train = []
for ratings in train_ratings:
    X_train.append(ratings[1][0])
    y_train.append(ratings[1][1])

X_train = np.array(X_train)
X_train = X_train.reshape(-1,1)

In [57]:
X_test = []
y_test = []
for ratings in test_ratings:
    X_test.append(ratings[1][0])
    y_test.append(ratings[1][1])

X_test = np.array(X_test)
X_test = X_test.reshape(-1,1)

In [58]:
X_test.shape

(10002, 1)

Train regressor on training data and make prediction on Test data<br>
Computed Mean Squared Error on predicted values

In [78]:
forest = RandomForestRegressor(max_depth=1, n_estimators=20)

rs = GradientBoostingRegressor(loss='ls', learning_rate=0.005, n_estimators=20)

reg= LinearRegression()

rs.fit(X_train, y_train)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

expected = y_test
predicted = rs.predict(X_test)

train_expected = y_train
train_predicted = rs.predict(X_train)

print("Training:\n%s" % np.sqrt(mean_squared_error(train_expected, train_predicted)))
print("Result:\n%s" % np.sqrt(mean_squared_error(expected, predicted)))

Training:
1.0317996903
Result:
1.03901005046


### Computed Ratings on missing values

In [65]:
missing_test = test_rdd.subtractByKey(test)

In [66]:
missing_ratings_user = missing_test.map(lambda x: ((x[0][0]), (x[0][1], x[1]))).join(avg_rating).map(lambda x: ((x[0], x[1][0][0]), (x[1][1], x[1][0][1])))

In [67]:
missing_ratings_user.take(5)

[]

Make predictions on test data using the trained regressor

In [79]:
predictions = test.map(lambda x: ((x[0]), (rs.predict(np.array(x[1][0]).reshape(1,-1))[0], x[1][1])))

In [80]:
final_predictions = predictions.union(missing_ratings_user)

In [81]:
final_predictions.count()

10002

In [147]:
mse = final_predictions.map(lambda x: (x[1][0]-x[1][1])**2)

In [148]:
np.sqrt(mse.mean())

1.0299346475243478

In [82]:
final_predictions = final_predictions.collect()

### Save predictions file

In [83]:
with open('/Users/lakshya/Desktop/INF-553/Project/TFIDF_CategoryBasedPredictions_pittsburgh.txt', 'w') as f:
    for item in final_predictions:
        f.write(str(item[0][0])+","+str(item[0][1])+","+str(item[1][0])+"\n")

### Making predictions on training data for Hybrid model

In [84]:
train_predict = train.map(lambda x: ((x[0]), (rs.predict(np.array(x[1][0]).reshape(1,-1))[0], x[1][1])))

In [85]:
train_predict.take(5)

[((u'cCke3VtFLtqgzkgYSrSE2g', u'BMR_AsSBzTQHqW-SQabI4w'),
  (3.8326497101895205, 4.0)),
 ((u'YE54kKTuqJJPNYWIKIpOEQ', u'RvwZqjdkZ_pER0moPXLZAQ'),
  (3.8326497101895205, 3.0)),
 ((u'4wp4XI9AxKNqJima-xahlg', u'ziJsGjXvidzZWC1I0-SOSg'),
  (3.828575120737566, 3.0)),
 ((u'U92V-gp13uMZL-sl_naTHA', u'zi6cB_bkswWPLD2k3IVtyg'),
  (3.828575120737566, 4.0)),
 ((u'rZQCd47n7OwPd71igVX6Og', u'IfHkboQZGPZkKpqHZCW4ag'),
  (3.8326497101895205, 1.0))]

In [166]:
train_predictions = train_predict.map(lambda x: (x[1][0]-x[1][1])**2)

In [167]:
np.sqrt(train_predictions.mean())

1.03320310263514

In [86]:
train_predict = train_predict.collect()

In [88]:
with open('/Users/lakshya/Desktop/INF-553/Project/TrainTFIDF_CategoryBasedPredictions_pittsburgh.txt', 'w') as f:
    for item in train_predict:
        f.write(str(item[0][0])+","+str(item[0][1])+","+str(item[1][0])+"\n")